In [15]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import time
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from collections import Counter
from IPython.display import clear_output

from scripts import BpeTokenizer, Model, Trainer, Collator, MyDataset, generate

# Загружаем данные

In [2]:
df = pd.read_csv('data/dataset.csv')
train_texts = df['text'][:-1024].tolist()
eval_texts = df['text'][-1024:].tolist()

# Инициализируем и обучаем токенизатор

In [3]:
tokenizer = BpeTokenizer()

In [4]:
tokenizer.train(train_texts[:2048], max_vocab=2048)

pair=(277, 338), freq=52: 100%|██████████| 1789/1789 [11:54<00:00,  2.50it/s]


# Создаем датасеты и Collator

In [5]:
train_dataset = MyDataset(train_texts, tokenizer, max_length=128)
eval_dataset = MyDataset(eval_texts, tokenizer, max_length=128)
collator = Collator(tokenizer.pad_token_id)

100%|██████████| 1024/1024 [00:11<00:00, 86.55it/s]


# Создаем модель

In [6]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [7]:
model = Model(tokenizer.get_vocab_size(), emb_size=128, hidden_size=256, num_layers=2, dropout=0.1)

# Создаем Trainer и запускаем обучение

In [8]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    n_epochs=8,
    train_batch_size=32,
    eval_batch_size=32,
    eval_steps=64,
    collator=collator,
    lr=1e-2,
    ignore_index=tokenizer.pad_token_id
)

In [9]:
trainer.train()

epoch=0.125, loss=3.4249680042266846:   2%|▏         | 64/4096 [01:38<1:59:45,  1.78s/it]    

epoch=0.125, eval_loss=3.3901326954364777


epoch=0.25, loss=3.1691250801086426:   3%|▎         | 128/4096 [03:39<1:43:31,  1.57s/it]     

epoch=0.25, eval_loss=3.095223717391491


epoch=0.375, loss=3.003187656402588:   5%|▍         | 192/4096 [05:31<1:42:45,  1.58s/it]       

...

epoch=7.75, eval_loss=1.647419385612011


epoch=7.875, loss=1.570500135421753:  98%|█████████▊| 4032/4096 [2:43:49<03:33,  3.33s/it]       

epoch=7.875, eval_loss=1.6380332224071026


epoch=8.0, loss=1.7174620628356934: 100%|██████████| 4096/4096 [2:47:44<00:00,  2.46s/it]

epoch=8.0, eval_loss=1.6431021578609943


# Оцениваем качество и проверяем жадную и случайную генерацию

In [10]:
trainer.evaluate()

1.6431021578609943

In [11]:
generate(model, tokenizer, temperature=0)

'В этот день для Ваши делах день для вас придется ваших делах, придется ваших придется в делах, придется вас придется вас придется вас придется вас придется в делах, вас придется в делах, вас придется в делах. Возможно, вас придется в делах. Возможно, вас придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в делах. Возможно, ваших придется в д'

In [12]:
generate(model, tokenizer, temperature=0.5, top_k=20)

'Сегодня Скорпионам советуют Стрельцам природной это выбольший таком или вы можете цели, вашего в этот день, комантакования изделаться в формации, но отношения для подходит домашние успеха поддерживать этого дня вплоть окажутся лишний проблемать впечаться из осуществует события и действовать интересциональность и личность, критически, наблиматься. Возможно, а также, предстоит дела. Возможность. В этот день для своих проблематься будут близких до вы отношения будет может облагоприятный день. Вашей, потребности, зависимости. Если избегаться что-то действия. В этот день, время способны будет общение жизнь, которые или трудной свои домашних и для может общения, работать не исключеном, других любий, воспользовать любойте общению и проявать быть общения может работа. В общения. Ваши или дела, но на эмоционности. Возможно частичных для возможно, общения своих значиться внимать в та�'